In [1]:
%load_ext lab_black

In [2]:
from pathlib import Path

import pandas as pd

In [3]:
data_dir = Path("./data")
assert data_dir.exists()
gene_node_file = data_dir / "gene_node.csv"
assert gene_node_file.exists()
gwas_node_file = data_dir / "gwas_node.csv"
assert gwas_node_file.exists()
edge_file = data_dir / "mrres.csv"
assert edge_file.exists()

# gene nodes

In [4]:
gene_node_raw = pd.read_csv(gene_node_file)
print(gene_node_raw.info())
gene_node_raw

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1525 entries, 0 to 1524
Data columns (total 4 columns):
X_id           1525 non-null object
X_name         1525 non-null object
chr            1525 non-null int64
description    1525 non-null object
dtypes: int64(1), object(3)
memory usage: 47.8+ KB
None


,X_id,X_name,chr,description
0,ENSG00000168685,IL7R,5,Interleukin-7 receptor subunit alpha;SeqId_508...
1,ENSG00000146701,MDH2,7,Malate dehydrogenase
2,ENSG00000023892,DEF6,6,Differentially expressed in FDCP 6 homolog;Seq...
3,ENSG00000109854,HTATIP2,11,Oxidoreductase HTATIP2;SeqId_10630_5;Q9BUP3
4,ENSG00000174080,CTSF,11,Cathepsin F;SeqId_9212_22;Q9UBX1
...,...,...,...,...
1520,ENSG00000177191,B3GNT8,19,UDP-GlcNAc:betaGal beta-1
1521,ENSG00000108602,ALDH3A1,17,Aldehyde dehydrogenase
1522,ENSG00000148180,GSN,9,Gelsolin;SeqId_16607_78;P06396
1523,ENSG00000164144,ARFIP1,4,Arfaptin-1;SeqId_13488_3;P53367


In [5]:
gene_node = gene_node_raw.dropna()
print(gene_node.info())
gene_node

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1525 entries, 0 to 1524
Data columns (total 4 columns):
X_id           1525 non-null object
X_name         1525 non-null object
chr            1525 non-null int64
description    1525 non-null object
dtypes: int64(1), object(3)
memory usage: 59.6+ KB
None


,X_id,X_name,chr,description
0,ENSG00000168685,IL7R,5,Interleukin-7 receptor subunit alpha;SeqId_508...
1,ENSG00000146701,MDH2,7,Malate dehydrogenase
2,ENSG00000023892,DEF6,6,Differentially expressed in FDCP 6 homolog;Seq...
3,ENSG00000109854,HTATIP2,11,Oxidoreductase HTATIP2;SeqId_10630_5;Q9BUP3
4,ENSG00000174080,CTSF,11,Cathepsin F;SeqId_9212_22;Q9UBX1
...,...,...,...,...
1520,ENSG00000177191,B3GNT8,19,UDP-GlcNAc:betaGal beta-1
1521,ENSG00000108602,ALDH3A1,17,Aldehyde dehydrogenase
1522,ENSG00000148180,GSN,9,Gelsolin;SeqId_16607_78;P06396
1523,ENSG00000164144,ARFIP1,4,Arfaptin-1;SeqId_13488_3;P53367


In [7]:
print(gene_node["X_id"].drop_duplicates().pipe(len))
print(gene_node["X_name"].drop_duplicates().pipe(len))

1477
1477


In [8]:
dup_genes = (
    gene_node[["X_id", "X_name"]]
    .groupby("X_id")
    .size()
    .reset_index(name="counts")
    .query("counts > 1")
)
print(dup_genes.head())
dup_df = gene_node[gene_node["X_id"].isin(dup_genes["X_id"])]
dup_df.sort_values("X_id")

                X_id  counts
1    ENSG00000002549       2
41   ENSG00000026103       2
44   ENSG00000028137       2
146  ENSG00000085514       2
278  ENSG00000105376       2


,X_id,X_name,chr,description
815,ENSG00000002549,LAP3,4,Cytosol aminopeptidase;SeqId_15610_72;P28838
814,ENSG00000002549,LAP3,4,Cytosol aminopeptidase;SeqId_13959_7;P28838
1010,ENSG00000026103,FAS,10,Tumor necrosis factor receptor superfamily mem...
579,ENSG00000026103,FAS,10,Tumor necrosis factor receptor superfamily mem...
1175,ENSG00000028137,TNFRSF1B,1,Tumor necrosis factor receptor superfamily mem...
...,...,...,...,...
512,ENSG00000196136,SERPINA3,14,Alpha-1-antichymotrypsin complex;SeqId_4153_11...
1096,ENSG00000257017,HP,16,Haptoglobin;SeqId_7905_30;P00738
866,ENSG00000257017,HP,16,Haptoglobin;SeqId_3054_3;P00738
901,ENSG00000274736,CCL23,17,C-C motif chemokine 23;SeqId_2913_1;P55773


# gwas nodes

In [9]:
gwas_node_raw = pd.read_csv(gwas_node_file)
print(gwas_node_raw.info())
gwas_node_raw

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 16 columns):
_id            16 non-null object
_name          16 non-null object
consortium     16 non-null object
build          16 non-null object
ncase          16 non-null int64
ncontrol       16 non-null int64
sample_size    16 non-null int64
nsnp           16 non-null int64
population     16 non-null object
sex            16 non-null object
author         16 non-null object
year           16 non-null int64
unit           16 non-null object
mr             16 non-null int64
category       16 non-null object
trait          16 non-null object
dtypes: int64(6), object(10)
memory usage: 2.1+ KB
None


,_id,_name,consortium,build,ncase,ncontrol,sample_size,nsnp,population,sex,author,year,unit,mr,category,trait
0,gbmi-a-00001-afr-b,Asthma,GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,5054,27599,32653,24660533,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Asthma
1,gbmi-a-00001-nfe-b,Asthma,GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,100736,1118479,1219215,47156680,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Asthma
2,gbmi-a-00002-afr-b,Chronic obstructive pulmonary disease (COPD),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,1978,27699,29677,23889093,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Chronic obstructive pulmonary disease (COPD)
3,gbmi-a-00002-nfe-b,Chronic obstructive pulmonary disease (COPD),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,51231,749346,800577,45532064,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Chronic obstructive pulmonary disease (COPD)
4,gbmi-a-00003-afr-b,Gout,GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,1313,33896,35209,25056505,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Gout
5,gbmi-a-00003-nfe-b,Gout,GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,20512,822995,843507,47218578,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Gout
6,gbmi-a-00004-afr-b,Heart Failure (HF),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,970,16823,17793,24631161,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Heart Failure (HF)
7,gbmi-a-00004-nfe-b,Heart Failure (HF),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,27064,769538,796602,46480843,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Heart Failure (HF)
8,gbmi-a-00005-afr-b,Idiopathic pulmonary fibrosis (IPF),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,168,8364,8532,18058634,African,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Idiopathic pulmonary fibrosis (IPF)
9,gbmi-a-00005-nfe-b,Idiopathic pulmonary fibrosis (IPF),GBMI (Global biobank Meta-analysis initiative),HG38/GRCh38,4066,747896,751962,45621626,European,Males/Females,Wei Zhou,2021,log odds ratio,1,Binary,Idiopathic pulmonary fibrosis (IPF)


In [11]:
print(gwas_node_raw["_id"].drop_duplicates().pipe(len))
print(gwas_node_raw["_name"].drop_duplicates().pipe(len))

16
8


# edge

In [12]:
edge_raw = pd.read_csv(edge_file)
print(edge_raw.info())
edge_raw

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19582 entries, 0 to 19581
Data columns (total 20 columns):
X_id           19582 non-null object
X_name         19582 non-null object
chr            19582 non-null int64
protein        19582 non-null object
uniprot        19582 non-null object
description    19582 non-null object
ancestry       19582 non-null object
disease        19582 non-null object
method         19582 non-null object
nsnp           19582 non-null int64
b              19582 non-null float64
ci_low         14945 non-null float64
ci_upp         14945 non-null float64
se             19582 non-null float64
pval           19582 non-null float64
selection      19582 non-null object
moescore       19582 non-null int64
ldcheck        1224 non-null object
pwcoco         1224 non-null object
pleiotropy     9656 non-null object
dtypes: float64(5), int64(3), object(12)
memory usage: 3.0+ MB
None


,X_id,X_name,chr,protein,uniprot,description,ancestry,disease,method,nsnp,b,ci_low,ci_upp,se,pval,selection,moescore,ldcheck,pwcoco,pleiotropy
0,ENSG00000168685,IL7R,5,IL-7 Ra,P16871,SeqId_5089_11;Interleukin-7 receptor subunit a...,European,gbmi-a-00001-nfe-b,Wald_ratio,1,0.135224,NaN,NaN,0.046594,3.705908e-03,DF+HF,1,True,False,NaN
1,ENSG00000146701,MDH2,7,Malate dehydrogenase 2,P40926,SeqId_15534_26;Malate dehydrogenase/ mitochond...,European,gbmi-a-00001-nfe-b,Wald_ratio,1,0.066043,NaN,NaN,0.034404,5.490592e-02,DF+HF,1,NaN,NaN,NaN
2,ENSG00000023892,DEF6,6,DEFI6,Q9H4E7,SeqId_14090_23;Differentially expressed in FDC...,European,gbmi-a-00001-nfe-b,Wald_ratio,1,-0.256950,NaN,NaN,0.051617,6.430000e-07,DF+HF,1,True,True,NaN
3,ENSG00000109854,HTATIP2,11,HTAI2,Q9BUP3,SeqId_10630_5;Oxidoreductase HTATIP2,European,gbmi-a-00001-nfe-b,Wald_ratio,1,-0.035607,NaN,NaN,0.017421,4.096695e-02,DF+HF,1,False,False,NaN
4,ENSG00000174080,CTSF,11,CATF,Q9UBX1,SeqId_9212_22;Cathepsin F,European,gbmi-a-00001-nfe-b,Wald_ratio,1,-0.014405,NaN,NaN,0.020015,4.717140e-01,DF+HF,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19577,ENSG00000120256,LRP11,6,LRP11,Q86VZ4,SeqId_15472_16;Low-density lipoprotein recepto...,African,gbmi-a-00004-afr-b,generalized IVW,8,0.123291,-0.066526,0.038228,0.048226,1.057282e-02,DF+HF,1,False,False,False
19578,ENSG00000105376,ICAM5,19,sICAM-5,Q9UMF0,SeqId_8245_27;Intercellular adhesion molecule 5,African,gbmi-a-00004-afr-b,generalized IVW,2,-0.085452,-0.062592,0.109604,0.078178,2.743783e-01,DF+HF,1,NaN,NaN,NaN
19579,ENSG00000028137,TNFRSF1B,1,TNF sR-II,P20333,SeqId_8368_102;Tumor necrosis factor receptor ...,African,gbmi-a-00004-afr-b,generalized IVW,3,0.047461,NaN,NaN,0.095236,6.182397e-01,DF+HF,1,NaN,NaN,False
19580,ENSG00000274736,CCL23,17,MPIF-1,P55773,SeqId_2913_1;C-C motif chemokine 23,African,gbmi-a-00004-afr-b,generalized IVW,3,0.110918,-0.089456,0.067650,0.084080,1.871027e-01,DF+HF,1,NaN,NaN,False


In [20]:
dup_edge_check = (
    edge_raw.groupby(["X_id", "disease"])
    .size()
    .reset_index(name="counts")
    .query("counts > 1")
)
dup_edge_check

,X_id,disease,counts


In [17]:
dup_gene_ids = dup_df["X_id"].drop_duplicates()
foobar = edge_raw[
    (edge_raw["X_id"].isin(dup_gene_ids))
    & (edge_raw["disease"] == "gbmi-a-00001-nfe-b")
]
print(foobar.info())
print(foobar["X_id"].drop_duplicates().pipe(len))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 32 to 9704
Data columns (total 20 columns):
X_id           46 non-null object
X_name         46 non-null object
chr            46 non-null int64
protein        46 non-null object
uniprot        46 non-null object
description    46 non-null object
ancestry       46 non-null object
disease        46 non-null object
method         46 non-null object
nsnp           46 non-null int64
b              46 non-null float64
ci_low         36 non-null float64
ci_upp         36 non-null float64
se             46 non-null float64
pval           46 non-null float64
selection      46 non-null object
moescore       46 non-null int64
ldcheck        8 non-null object
pwcoco         8 non-null object
pleiotropy     32 non-null object
dtypes: float64(5), int64(3), object(12)
memory usage: 7.5+ KB
None
46
